## Requests

In [1]:
import pandas as pd
import random
import os, time
from datetime import datetime 

import requests
import json
from bs4 import BeautifulSoup

### 變數

In [2]:
#不完整
input_city_nm = input('請輸入縣市名稱：')
if input_city_nm=='基隆市':
    city_num = 2
    #section = 'all'
elif input_city_nm=='台北市':
    city_num = 1
elif input_city_nm=='新北市':
    city_num = 3
    #section = [37,38]
elif input_city_nm=='桃園市':
    city_num = 6
elif input_city_nm=='新竹市':
    city_num = 4
elif input_city_nm=='台中市':
    city_num = 8
elif input_city_nm=='台南市':
    city_num = 15
    #section = [206,207,208,209,210,211]
elif input_city_nm=='高雄市':
    city_num = 17
else:
    print('請確認輸入的縣市是否正確')

請輸入縣市名稱：高雄市


In [3]:
min_time = 5
max_time = 10

### 紀錄 

* Cookie 取得 X-CSRF-TOKEN
* 總筆數及頁數

In [4]:
def search(result=[], filter_params=None, sort_params=None, input_city_nm='台北市', city_num=1, page_num=0, want_page=1, min_time=2, max_time=5):
    
    # 紀錄 Cookie 取得 X-CSRF-TOKEN
    s = requests.Session()
    url = 'https://rent.591.com.tw/'
    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'
    }
    
    r = s.get(url, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    token_item = soup.select_one('meta[name="csrf-token"]')
    #headers = self.headers.copy()
    headers['X-CSRF-TOKEN'] = token_item.get('content')
    #print(headers)
    
    
    # 搜尋房屋
    url_search = 'https://rent.591.com.tw/home/search/rsList'
    params = 'is_format_data=1&is_new_list=1&type=1'

    # 篩選參數
    if filter_params:
        # 加上篩選參數，要先轉換為 URL 參數字串格式
        params += ''.join([f'&{key}={value}' for key, value, in filter_params.items()])
    else:
        params += r'&region={}'.format(city_num)
    
    # 在 cookie 設定地區縣市，避免某些條件無法取得資料
    # 類似操作 dict：可以添加cookie，不會清除原cookie
    #s.cookies['urlJumpIp'] = str(city_num)
    # requests.cookies.set() 函数：會清除原有cookie
    s.cookies.set('urlJumpIp', filter_params.get('region', str(city_num)) if filter_params else str(city_num), domain='.591.com.tw')
    #s.cookies.set('urlJumpIp', str(city_num), domain='.591.com.tw')

    # 排序參數
    if sort_params:
        params += ''.join([f'&{key}={value}' for key, value, in sort_params.items()])
 

    # 取得房屋詳情
    res = s.get(url_search, params=params, headers=headers)
    if res.status_code != requests.codes.ok:
        print('請求失敗', res.status_code)
        
    json_data = res.json()
    tot_num = int(json_data['records'].replace(',',''))     #總筆數
    want_page_num = int(tot_num/30)+1
    
    if want_page==None:
        want_page = want_page_num
    #print('總筆數：{}'.format(tot_num))
    #print('總頁數：{}'.format(want_page))
    
    while page_num < want_page:

        params_page = params + f'&firstRow={page_num*30}'

        res = s.get(url_search, params=params_page, headers=headers)
        json_data = res.json()
        all_data = json_data['data']['data']

        for i in range(0, 30):
            data_num = i+1
            print('第 {} 筆'.format(data_num))
            #print('縣市：{}'.format(input_city_nm))
            section_nm = all_data[i]['section_name']
            #print('行政區：{}'.format(section_nm))
            location = all_data[i]['location'].replace('-','')
            #print('地址：{}'.format(location))

            role_nm = all_data[i]['role_name']
            #print('角色：{}'.format(role_nm))
            contact = all_data[i]['contact']
            #print('姓名：{}'.format(contact))

            community = all_data[i]['community']
            #print('社區：{}'.format(community))
            kind_nm = all_data[i]['kind_name']
            #print('類型：{}'.format(kind_nm))
            area = all_data[i]['area']
            #print('坪數：{}'.format(area))
            floor_str = all_data[i]['floor_str']
            #print('樓層：{}'.format(floor_str))
            price = all_data[i]['price'].replace(',','') #+all_data[i]['price_unit']
            #print('金額：{}'.format(price))

            title = all_data[i]['title']
            #print('主旨：{}'.format(title))
            post_id = all_data[i]['post_id']
            #print('網址：{}'.format(all_data[i]['post_id']))
            
            result.append(
                           dict(data_num=data_num, input_city_nm=input_city_nm, section_nm=section_nm, location=location, 
                                role_nm=role_nm, contact=contact, 
                                community=community, kind_nm=kind_nm, area=area, floor_str=floor_str, price=price,
                                title=title, post_id=post_id))
            
            #print('='*30)
        page_num += 1

        time.sleep(random.randint(min_time, max_time) + abs(random.random()))
    return(dict(result=result, want_page_num=want_page_num))
    #return(result)

In [5]:
def get_house_detail(result, result_detail=[], result_error=[], min_time=2, max_time=5):
    
    node_data = pd.DataFrame(result)
    node_data = node_data.rename(columns={'data_num':'序號', 'input_city_nm':'縣市', 'section_nm':'行政區', 'location':'地址', 'role_nm':'角色', 
                                      'contact':'姓名', 'community':'社區', 'kind_nm':'類型', 'area':'坪數', 'floor_str':'樓層',
                                      'price':'金額', 'title':'主旨', 'post_id':'網址'})

    for post_id in node_data['網址']:
        try:
            s = requests.Session()
            print('網址：{}'.format(post_id))
            url = f'https://rent.591.com.tw/home/{post_id}'

            headers = {
                'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'
            }

            r = s.get(url, headers=headers)
            soup = BeautifulSoup(r.text, 'html.parser')
            # print(soup)
            token_item = soup.select_one('meta[name="csrf-token"]')

            headers['X-CSRF-TOKEN'] = token_item.get('content')
            headers['deviceid'] = s.cookies.get_dict()['T591_TOKEN']
            headers['device'] = 'pc'
            # headers['token'] = s.cookies.get_dict()['PHPSESSID']

            url = f'https://bff.591.com.tw/v1/house/rent/detail?id={post_id}'
            res = s.get(url, headers=headers)
            house_detail = res.json()['data']

            room_str = house_detail['info'][0]['value']
            #print('格局：{}'.format(room_str))

            kind_nm = house_detail['info'][3]['value']
            #print('型態：{}'.format(kind_nm))

            phone = house_detail['linkInfo']['mobile']
            #print('電話：{}'.format(phone))

            now_time = datetime.now().strftime('%y/%m/%d %H:%M:%S')
            #print('時間：{}'.format(now_time))

            result_detail.append(dict(post_id=post_id, room_str=room_str, kind_nm=kind_nm, phone=phone, now_time=now_time))
            #print('='*30)

        except Exception as e:
            print('網址：{}，異常訊息：{}'.format(post_id, e))
            result_error.append(dict(post_id=post_id, error=e))
            pass
        
        time.sleep(random.randint(min_time, max_time) + abs(random.random()))
    print('='*30)
    
    return(dict(result=result_detail, error=result_error))

In [6]:
result_search_tmp = search(filter_params=None, sort_params=None, result=[], input_city_nm=input_city_nm, city_num=city_num, page_num=0, want_page=1, min_time=min_time, max_time=max_time)
want_page_record = result_search_tmp['want_page_num']
print('='*30)

result = []
result_detail = []
result_error = []
#page_num =0

for page_num in range(0, 2): 
    print('第 {} 頁開始'.format(page_num+1))
    want_page = page_num + 1
    #result_search_tmp = []
    result_search_tmp = search(filter_params=None, sort_params=None, result=[], input_city_nm=input_city_nm, city_num=city_num, page_num=page_num, want_page=want_page, min_time=min_time, max_time=max_time)
    for index_num in range(0, len(result_search_tmp['result'])):
        result_search_tmp['result'][index_num]['page'] = page_num+1
        #dict 如何合併 python
        result.append(result_search_tmp['result'][index_num])
    
    #result_detail_tmp = []
    result_detail_tmp = get_house_detail(result_search_tmp['result'], [], [], min_time, max_time)
    for index_num in range(0, len(result_detail_tmp['result'])):
        result_detail.append(result_detail_tmp['result'][index_num])
    for index_num in range(0, len(result_detail_tmp['error'])): 
        result_error.append(result_detail_tmp['error'][index_num])
 
    print('總頁數：{}'.format(want_page_record))
    print('='*30)

第 1 筆
第 2 筆
第 3 筆
第 4 筆
第 5 筆
第 6 筆
第 7 筆
第 8 筆
第 9 筆
第 10 筆
第 11 筆
第 12 筆
第 13 筆
第 14 筆
第 15 筆
第 16 筆
第 17 筆
第 18 筆
第 19 筆
第 20 筆
第 21 筆
第 22 筆
第 23 筆
第 24 筆
第 25 筆
第 26 筆
第 27 筆
第 28 筆
第 29 筆
第 30 筆
第 1 頁開始
第 1 筆
第 2 筆
第 3 筆
第 4 筆
第 5 筆
第 6 筆
第 7 筆
第 8 筆
第 9 筆
第 10 筆
第 11 筆
第 12 筆
第 13 筆
第 14 筆
第 15 筆
第 16 筆
第 17 筆
第 18 筆
第 19 筆
第 20 筆
第 21 筆
第 22 筆
第 23 筆
第 24 筆
第 25 筆
第 26 筆
第 27 筆
第 28 筆
第 29 筆
第 30 筆
網址：10907735
網址：12287277
網址：3566998
網址：12028101
網址：12674257
網址：12899606
網址：13392708
網址：13321525
網址：13250468
網址：13377400
網址：13350134
網址：13381130
網址：13243961
網址：13364188
網址：13258136
網址：13381609
網址：13359482
網址：13370676
網址：13290640
網址：13251155
網址：13366681
網址：13327035
網址：13361715
網址：13247903
網址：13282386
網址：13391951
網址：13329236
網址：13260022
網址：13342537
網址：13153164
總頁數：170
第 2 頁開始
第 1 筆
第 2 筆
第 3 筆
第 4 筆
第 5 筆
第 6 筆
第 7 筆
第 8 筆
第 9 筆
第 10 筆
第 11 筆
第 12 筆
第 13 筆
第 14 筆
第 15 筆
第 16 筆
第 17 筆
第 18 筆
第 19 筆
第 20 筆
第 21 筆
第 22 筆
第 23 筆
第 24 筆
第 25 筆
第 26 筆
第 27 筆
第 28 筆
第 29 筆
第 30 筆
網址：13361028
網址

In [7]:
node_df = pd.DataFrame(result)
node_df = node_df.rename(columns={'data_num':'序號', 'input_city_nm':'縣市', 'section_nm':'行政區', 'location':'地址', 'role_nm':'角色', 
                                  'contact':'姓名', 'community':'社區', 'kind_nm':'類型', 'area':'坪數', 'floor_str':'樓層',
                                  'price':'金額', 'title':'主旨', 'post_id':'網址', 'page':'頁碼'})
node_df.head()
#node_df.shape

,序號,縣市,行政區,地址,角色,姓名,社區,類型,坪數,樓層,金額,主旨,網址,頁碼
0,1,高雄市,苓雅區,苓雅區六合路31號,屋主,柯先生,,獨立套房,5,3F/5F,9000,電梯套房/免追垃圾車/可開發票/申請補助,10907735,1
1,2,高雄市,左營區,左營區民族一路,屋主,邱小姐,榮總、高鐵,整層住家,28,2F/14F,13000,幸運屋有與社會住宅區合作,12287277,1
2,3,高雄市,苓雅區,苓雅區凱旋二路31巷,屋主,黃小姐,,獨立套房,5,5F/5F,6000,近高師大長青學苑全新套房(機車停車位),3566998,1
3,4,高雄市,小港區,小港區松福街15號,屋主,黃先生,東坡居套房出租,獨立套房,5,3F/5F,6300,高雄餐旅大學旁走路上學(東坡居學生套房),12028101,1
4,5,高雄市,前鎮區,前鎮區中山二路,屋主,劉小姐,新海灣,整層住家,113.2,12F/29F,60000,富貴寶宅！環境好！採光視野佳！,12674257,1


In [8]:
node_detail = pd.DataFrame(result_detail)
node_detail = node_detail.rename(columns={'post_id':'網址', 'room_str':'格局', 'kind_nm':'型態', 'phone':'電話', 'now_time':'時間'})
node_detail.head()
#node_detail.shape

,網址,格局,型態,電話,時間
0,10907735,獨立套房,公寓,0972-192-190,22/10/14 14:00:15
1,12287277,2房2廳1衛,電梯大樓,0936-419-299,22/10/14 14:00:23
2,3566998,獨立套房,公寓,0952-991-511,22/10/14 14:00:33
3,12028101,獨立套房,電梯大樓,0925-081-751,22/10/14 14:00:45
4,12674257,4房2廳3衛,電梯大樓,0938-519-666,22/10/14 14:00:53


In [9]:
#final_df_temp = node_df.merge(node_detail, how='inner', on=['網址'])
#final_df = final_df.append(final_df_temp , ignore_index=True)
final_df = node_df.merge(node_detail, how='inner', on=['網址'])
final_df.shape

(62, 18)

In [10]:
result_error

[]

### 資料輸出

#### CSV檔

In [15]:
path = 'D:\GitHub_彙整\GitHub_2022_爬蟲_591租屋\Data'

In [12]:
final_df.to_csv(r'{}\591租屋_{}_完整.csv'.format(path, input_city_nm), index=False, sep='^')

#### Excel檔

In [13]:
Result_PATH = r'{}\591租屋_{}_完整.xlsx'.format(path, input_city_nm)

# writing to Excel
student_result  = pd.ExcelWriter( Result_PATH, engine='xlsxwriter')

# write students data to excel
final_df.to_excel(student_result, sheet_name=input_city_nm, index=False, encoding='utf8')

# save the students result excel
student_result.save()
print('Students data is successfully written into Excel File')

Students data is successfully written into Excel File


#### 依條件篩選資料

In [16]:
df1 = pd.read_excel(r'{}\系統台分區.xlsx'.format(path))

In [17]:
df2 = final_df.merge(df1, how='inner', on=['縣市','行政區'])
df2.head()

,序號,縣市,行政區,地址,角色,姓名,社區,類型,坪數,樓層,金額,主旨,網址,頁碼,格局,型態,電話,時間,系統,系統台區
0,1,高雄市,苓雅區,苓雅區六合路31號,屋主,柯先生,,獨立套房,5,3F/5F,9000,電梯套房/免追垃圾車/可開發票/申請補助,10907735,1,獨立套房,公寓,0972-192-190,22/10/14 14:00:15,D,DD
1,3,高雄市,苓雅區,苓雅區凱旋二路31巷,屋主,黃小姐,,獨立套房,5,5F/5F,6000,近高師大長青學苑全新套房(機車停車位),3566998,1,獨立套房,公寓,0952-991-511,22/10/14 14:00:33,D,DD
2,3,高雄市,苓雅區,苓雅區凱旋二路31巷,屋主,黃小姐,,獨立套房,5,5F/5F,6000,近高師大長青學苑全新套房(機車停車位),3566998,1,獨立套房,公寓,0952-991-511,22/10/14 14:08:24,D,DD
3,26,高雄市,苓雅區,苓雅區凱旋二路31巷,屋主,黃小姐,,獨立套房,5,5F/5F,6000,近高師大長青學苑全新套房(機車停車位),3566998,2,獨立套房,公寓,0952-991-511,22/10/14 14:00:33,D,DD
4,26,高雄市,苓雅區,苓雅區凱旋二路31巷,屋主,黃小姐,,獨立套房,5,5F/5F,6000,近高師大長青學苑全新套房(機車停車位),3566998,2,獨立套房,公寓,0952-991-511,22/10/14 14:08:24,D,DD


In [18]:
df2.iloc[:, 1:15].drop_duplicates().to_excel(r'{}\591租屋_{}_需求.xlsx'.format(path, input_city_nm), index=False, encoding='utf8')

## 參考資料

* [591 租屋網 - 搜尋房屋與房屋詳情](https://blog.jiatool.com/posts/house591_spider/)
* [python requests库添加自定义cookie的方法](https://blog.csdn.net/weixin_39888943/article/details/111079997?utm_medium=distribute.pc_aggpage_search_result.none-task-blog-2~aggregatepage~first_rank_ecpm_v1~rank_v31_ecpm-5-111079997-null-null.pc_agg_new_rank&utm_term=python%20request%20%E5%A4%9A%E4%B8%AAcookie&spm=1000.2123.3001.4430)
* [3個實用的Pandas套件清理重複資料教學](https://www.learncodewithmike.com/2022/01/pandas-drop-duplicate-data.html)

In [15]:
post_id = 13322979

In [16]:
s = requests.Session()
print('網址：{}'.format(post_id))
url = f'https://rent.591.com.tw/home/{post_id}'

headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'
}

r = s.get(url, headers=headers)
soup = BeautifulSoup(r.text, 'html.parser')
# print(soup)
token_item = soup.select_one('meta[name="csrf-token"]')

headers['X-CSRF-TOKEN'] = token_item.get('content')
headers['deviceid'] = s.cookies.get_dict()['T591_TOKEN']
headers['device'] = 'pc'
# headers['token'] = s.cookies.get_dict()['PHPSESSID']

url = f'https://bff.591.com.tw/v1/house/rent/detail?id={post_id}'
res = s.get(url, headers=headers)
house_detail = res.json()['data']

room_str = house_detail['info'][0]['value']
#print('格局：{}'.format(room_str))

kind_nm = house_detail['info'][3]['value']
#print('型態：{}'.format(kind_nm))

phone = house_detail['linkInfo']['mobile']
#print('電話：{}'.format(phone))

now_time = datetime.now().strftime('%y/%m/%d %H:%M:%S')
#print('時間：{}'.format(now_time))

result_detail.append(dict(post_id=post_id, room_str=room_str, kind_nm=kind_nm, phone=phone, now_time=now_time))
#print('='*30)


網址：13322979
